<a href="https://colab.research.google.com/github/Lu13be/AmigoSecreto/blob/main/Copia_de_optimizaci_n_aeroportuaria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**CLASE 1**

In [ ]:
import pandas as pd

datos =  pd.read_csv('/content/flights.csv')
datos.shape

FileNotFoundError: [Errno 2] No such file or directory: '/content/flights.csv'

In [ ]:
datos.head()

In [ ]:
datos.info()

In [ ]:
datos.describe()

In [ ]:
datos.describe(include='O')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

avg_delay = datos.groupby('airline')['delay'].mean().reset_index()
sns.barplot(x='airline',y='delay',data=avg_delay)
plt.title('Compañías aéreas vs atraso promedio')
plt.xlabel('Compañía aérea')
plt.ylabel('Atraso promedio en minutos')
plt.show()

sns.countplot(data=datos, x='airline')
plt.title('Número de vuelos por Compañía aérea')
plt.xlabel('Compañía aérea')
plt.ylabel('Número de vuelos')
plt.show()

NameError: name 'datos' is not defined

In [ ]:
avg_delay = datos.groupby('schengen')['delay'].mean().reset_index()
sns.barplot(x='schengen',y='delay',data=avg_delay)
plt.title('Tipo de vuelo vs atraso promedio')
plt.xlabel('Tipo de vuelo')
plt.ylabel('Atraso promedio en minutos')
plt.show()

sns.countplot(data=datos, x='schengen')
plt.title('Número de vuelos por Tipo')
plt.xlabel('Tipo de Vuelo')
plt.ylabel('Número de vuelos')
plt.show()

In [ ]:
avg_delay = datos.groupby('is_holiday')['delay'].mean().reset_index()
sns.barplot(x='is_holiday',y='delay',data=avg_delay)
plt.title('Días feriados vs atraso promedio')
plt.xlabel('Días feriados')
plt.ylabel('Atraso promedio en minutos')
plt.show()

In [ ]:
datos['aircraft_type'].value_counts().index

In [ ]:
order = datos['aircraft_type'].value_counts().index
sns.countplot(data=datos, x='aircraft_type',order=order)
plt.title('Número de vuelos por Tipo de aeronave')
plt.xticks(rotation=70)
plt.xlabel('Tipo de Aeronave')
plt.ylabel('Número de vuelos')
plt.show()

In [ ]:
sns.histplot(data=datos,x='arrival_time',kde=True)

https://en.wikipedia.org/wiki/Freedman%E2%80%93Diaconis_rule

In [ ]:
sns.histplot(data=datos,x='arrival_time',kde=True,bins=10)

In [ ]:
import numpy as np

def ancho_bin(df,columna):
  q75,q25 = np.percentile(df[columna],[75,25])
  iqr = q75-q25
  ancho = 2 * iqr * np.power(len(df[columna]), -1/3)
  return ancho

In [ ]:
binwidth = ancho_bin(datos,'arrival_time')
sns.histplot(data=datos,x='arrival_time',kde=True,binwidth=binwidth)

In [ ]:
binwidth = ancho_bin(datos,'departure_time')
sns.histplot(data=datos,x='departure_time',kde=True,binwidth=binwidth)

In [ ]:
atraso_promedio = datos['delay'].mean()
atraso_mediana = datos['delay'].median()

fig, ax = plt.subplots(1,2,figsize=(9,4))

sns.boxplot(data=datos, y='delay',ax=ax[0])
ax[0].set_title('Boxplot')
ax[0].axhline(y=atraso_promedio, color='r', linestyle='--', label='Promedio')
ax[0].legend()

binwidth = ancho_bin(datos, 'delay')
sns.histplot(data=datos, x='delay', ax=ax[1], kde=True, binwidth=binwidth)
plt.ylabel('Número de vuelos')
plt.grid(False)
ax[1].set_title('Histograma')
ax[1].axvline(x=atraso_promedio, color='r', linestyle='--', label='Promedio')
ax[1].axvline(x=atraso_mediana, color='y', linestyle='--', label='Mediana')
ax[1].legend()

plt.tight_layout()

plt.show()

#**CLASE 2**

In [ ]:
datos.columns

In [ ]:
datos['date'] = datos['year'].astype(str) +'-'+(datos['day']+1).astype(str)
datos['date'] = pd.to_datetime(datos['date'],format='%Y-%j')
datos.tail(2)

In [ ]:
datos['is_weekend'] = datos['date'].dt.weekday.isin([5,6])
datos['day_name'] = datos['date'].dt.day_name()
datos.sample(5)

In [ ]:
datos.info()

In [ ]:
import warnings

warnings.filterwarnings('ignore')

In [ ]:
datos['schengen'] = datos['schengen'].replace({'non-schengen':0,'schengen':1})
datos['is_holiday'] = datos['is_holiday'].replace({False:0,True:1})
datos['is_weekend'] = datos['is_weekend'].replace({False:0,True:1})
datos.sample(3)

In [ ]:
categoricas = ['airline','aircraft_type','origin','day_name']

pd.get_dummies(data=datos,columns=categoricas, dtype=int).head()

In [ ]:
datos_codificados =pd.get_dummies(data=datos,columns=categoricas, dtype=int)
datos_codificados.sample(5)

In [ ]:
datos[['arrival_time','departure_time']].corr()

In [ ]:
datos_codificados.info()

In [ ]:
df = datos_codificados.drop(columns=['flight_id','departure_time','day','year','date'])
df.sample(10)

#**CLASE 3**

https://scikit-learn.org/stable/

In [ ]:
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split

X = df.drop(['delay'],axis=1)
y = df['delay']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42)

baseline = DummyRegressor()
baseline.fit(X_train,y_train)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

y_pred_dummy = baseline.predict(X_test)

def calcular_regresion(y_test,y_pred):
  rmse = mean_squared_error(y_test,y_pred)
  mae= mean_absolute_error(y_test,y_pred)
  r2 = r2_score(y_test,y_pred)
  metricas = {
      'RMSE':round(rmse**(1/2),4),
      'MAE': round(mae,4),
      'R2': round(r2,4)
  }
  return metricas

In [ ]:
resultados_baseline = calcular_regresion(y_test, y_pred_dummy)
resultados_baseline

In [ ]:
from sklearn.ensemble import RandomForestRegressor

modelo = RandomForestRegressor(max_depth=5, random_state=42)
modelo.fit(X_train,y_train)
ypred = modelo.predict(X_test)
resultados_rf = calcular_regresion(y_test,ypred)
resultados_rf

In [ ]:
from yellowbrick.regressor import prediction_error

visualizer = prediction_error(modelo, X_train, y_train, X_test, y_test);

In [ ]:
from yellowbrick.regressor import residuals_plot

viz = residuals_plot(modelo, X_train, y_train, X_test, y_test);

In [ ]:
from sklearn.model_selection import KFold, cross_validate

scoring = {
    'RMSE':'neg_root_mean_squared_error',
    'MAE':'neg_mean_absolute_error',
    'R2':'r2'
}

cv = KFold(n_splits=5,shuffle=True, random_state=42)
cv_results = cross_validate(modelo, X_train, y_train, scoring=scoring, cv=cv)
cv_results

#**CLASE 4**

In [ ]:
len(df.columns)

In [ ]:
importances = modelo.feature_importances_

In [ ]:
feature_importances = pd.DataFrame({'Features':X.columns,'Importances':(importances*100).round(2)}).sort_values('Importances',ascending=False)
feature_importances

In [ ]:
resultados = pd.DataFrame(index=['RMSE','MAE','R2'])
model_features = RandomForestRegressor(max_depth=5,random_state=42)
ct_features = [i if i != 0 else 1 for i in range(0,35,5)]

for i in ct_features:
  selected_features = feature_importances['Features'].values[:i]
  X_train_sel = X_train[selected_features]
  X_test_sel = X_test[selected_features]
  model_features.fit(X_train_sel, y_train)
  y_pred = model_features.predict(X_test_sel)
  metricas = calcular_regresion(y_test,y_pred)
  resultados[i] = list(metricas.values())

resultados

In [ ]:
resultados = pd.DataFrame(index=['RMSE','MAE','R2'])
model_features = RandomForestRegressor(max_depth=5,random_state=42)
ct_features = range(10,16)

for i in ct_features:
  selected_features = feature_importances['Features'].values[:i]
  X_train_sel = X_train[selected_features]
  X_test_sel = X_test[selected_features]
  model_features.fit(X_train_sel, y_train)
  y_pred = model_features.predict(X_test_sel)
  metricas = calcular_regresion(y_test,y_pred)
  resultados[i] = list(metricas.values())

resultados

In [ ]:
selected_features = feature_importances['Features'].values[:13]
X_selected_features = X[selected_features]
X_selected_features

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_selected_features,y,random_state=42)

In [ ]:
param_grid = {
              'max_depth':[5,10,15],
              'min_samples_leaf':[1,2,3],
              'min_samples_split':[2,4,6],
              'n_estimators':[100,150,200]
             }

In [ ]:
from sklearn.model_selection import GridSearchCV

cv = KFold(n_splits=5, shuffle=True, random_state=42)
model_grid = GridSearchCV(RandomForestRegressor(random_state=42),
                          param_grid=param_grid, scoring='r2',
                          cv=cv)

model_grid.fit(X_train,y_train)

In [ ]:
model_grid.best_params_

In [ ]:
y_pred_grid = model_grid.predict(X_test)
metricas_model_grid = calcular_regresion(y_test,y_pred_grid)
metricas_model_grid

In [ ]:
resultados['modelo_grid'] = list(metricas_model_grid.values())
resultados

In [ ]:
import pickle

try:
  with open('champion.pkl','wb') as file:
    pickle.dump(model_grid.best_estimator_,file)
  print('Modelo serializado con éxito')
except Exception as e:
  print('Se presentó un error al almacenar el modelo: ',str(e))

#1. Haga lo que hicimos

In [ ]:
import pandas as pd
datos = pd.read_csv('flights.csv')

In [ ]:
datos.describe()

In [ ]:
datos.describe(include='O')

In [ ]:
datos.info()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
retraso_promedio = datos.groupby('airline')['delay'].mean().reset_index()
sns.barplot(x='airline', y='delay', data=retraso_promedio)
plt.title('Aerolíneas vs retrasos promedio')
plt.xlabel('Aerolíneas')
plt.ylabel('Retraso promedio en minutos')
plt.show()

sns.countplot(data=datos, x='airline')
plt.title('Número de vuelos por aerolínea')
plt.xlabel('Aerolínea')
plt.ylabel('Número de vuelos')
plt.show()

In [ ]:
retraso_promedio = datos.groupby('schengen')['delay'].mean().reset_index()
sns.barplot(x='schengen', y='delay', data=retraso_promedio)
plt.title('Tipo de vuelo vs retrasos promedio')
plt.xlabel('Tipo de vuelo')
plt.ylabel('Retraso promedio en minutos')
plt.show()

sns.countplot(data=datos, x='schengen')
plt.title('Número de vuelos por tipo de vuelo')
plt.xlabel('Tipo de vuelo')
plt.ylabel('Número de vuelos')
plt.show()

In [ ]:
retraso_promedio = datos.groupby('is_holiday')['delay'].mean().reset_index()
sns.barplot(x='is_holiday', y='delay', data=retraso_promedio)
plt.title('Día festivo vs retrasos promedio')
plt.xlabel('¿Es día festivo?')
plt.ylabel('Retraso promedio en minutos')
plt.show()

In [ ]:
orden = datos['aircraft_type'].value_counts().index
sns.countplot(data=datos, x='aircraft_type', order=orden)
plt.title('Número de vuelos por tipo de aeronave')
plt.xticks(rotation=70)
plt.xlabel('Tipo de aeronave')
plt.ylabel('Número de vuelos')
plt.show()

In [ ]:
import numpy as np

def calcular_ancho_bin(df, columna):
    Q75, Q25 = np.percentile(df[columna], [75, 25])
    IQR = Q75 - Q25
    ancho_bin = 2 * IQR * np.power(len(df[columna]), -1/3)
    return ancho_bin

In [ ]:
ancho_bin_at = calcular_ancho_bin(datos, 'arrival_time')
ancho_bin_dt = calcular_ancho_bin(datos, 'departure_time')
sns.histplot(data=datos, x='arrival_time', kde=True, binwidth=ancho_bin_at)
sns.histplot(data=datos, x='departure_time', kde=True, binwidth=ancho_bin_dt)
Copia el código

In [ ]:
media_retraso = datos['delay'].mean()
mediana_retraso = datos['delay'].median()

fig, axes = plt.subplots(1, 2, figsize=(9, 4))

sns.boxplot(data=datos, y='delay', ax=axes[0])
axes[0].set_title('Boxplot')

axes[0].axhline(y=media_retraso, color='r', linestyle='--', label='Media')
axes[0].legend()

ancho_bin_delay = calcular_ancho_bin(datos, 'delay')
sns.histplot(data=datos, x='delay', ax=axes[1], kde=True, binwidth=ancho_bin_delay)
plt.ylabel('Número de vuelos')
plt.grid(False)
axes[1].set_title('Histograma')

axes[1].axvline(x=media_retraso, color='r', linestyle='--', label='Media')
axes[1].axvline(x=mediana_retraso, color='y', linestyle='--', label='Mediana')
axes[1].legend()

plt.tight_layout()
plt.show()

#2. Haga lo que hicimos

In [ ]:
datos['date'] = pd.to_datetime(datos[datos['year'].astype(str)+ '-' + (datos['day']+1).astype(str), format='%Y-%J')

datos['is_weekend'] = datos['date'].dt.weekday.isin([5, 6])

datos['day_name'] = datos['date'].dt.day_name()

In [ ]:
datos['schegen'] = datos['schengen'].replace({'non-schengen': 0, 'schengen': 1})
datos['is_holiday'] = datos['is_holiday'].replace({False: 0, True: 1})
datos['is_weekend'] = datos['is_weekend'].replace({False: 0, True: 1})

In [ ]:
variables_categoricas = ['airline', 'aircraft_type', 'origin', 'day_name']
df_encoded = pd.get_dummies(data=datos, columns=variables_categoricas, dtype=int)


In [ ]:
df_clean = df_encoded.drop(['flight_id', 'departure_time', 'day', 'year', 'date'], axis=1)

#3. Haga lo que hicimos

In [ ]:
from sklearn.model_selection import train_test_split

x = df_clean.drop(['delay'], axis=1)
y = df_clean['delay']

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)

In [ ]:
from sklearn.dummy import DummyRegressor

model_dummy = DummyRegressor()
model_dummy.fit(x_train, y_train)
y_pred_dummy = model_dummy.predict(x_test)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred,)
r2 = r2_score(y_test, y_pred)

metricas = {
    'Raíz del Error Cuadrático Medio': round(rmse, 4),
    'Error Absoluto Medio': round(mae, 4),
    'R2 Score': round(r2, 4)
}

return metricas


In [ ]:
calcular_metricas_regresion(y_test, y_pred_dummy)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model_rf = RandomForestRegressor(max_depth=5, random_state=42)
model_rf.fit(x_train, y_train)
y_pred = model_rf.predict(x_test)
calcular_metricas_regresion(y_test, y_pred_rf)


In [ ]:
from yellowbrick.regressor import prediction_error

visualizer = prediction_error(model_rf, x_train, y_train, x_test, y_test)


In [ ]:
from yellowbrick.regressor import residuals_plot

viz = residuals_plot(model_rf, x_train, y_train, x_test, y_test)

In [ ]:
from sklearn.model_selection import KFold, cross_validate

scoring = {
     'mae': 'neg_mean_absolute_error',
     'rmse': 'neg_root_mean_squared_error',
     'R2': 'r2'
}

cv = KFold(n_splits=5, shuffle=True, random_state=42)
cv_results = cross_validate(model_rf, x_train, y_train, cv=cv, scoring=scoring)

for metric in scoring.keys():
  scores = cv_results[f'test_{metric}']
  mean_score = "{:.3f}".format(Scores.mean())
  std_score = "{:.3f}".format(Scores.std())
  print(f"{metric.upper()} Scores: {[f'{val:.3f]' for val in scores]}")
  print(f"{metric.upper()}   Medida: {mean_score}, Std: {std_score}")
  print('--------------------------------------------------------------------')

#Hago lo que hicimos

In [4]:
import pandas as pd
datos = pd.read_csv('flights.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'flights.csv'

In [5]:
datos.describe()

NameError: name 'datos' is not defined

In [6]:
datos.describe(include='O')

NameError: name 'datos' is not defined

In [7]:
datos.info()

NameError: name 'datos' is not defined

In [8]:
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
retraso_promedio = datos.groupby('airline')['delay'].mean().reset_index()
sns.barplot(x='airline', y='delay', data=retraso_promedio)
plt.title('Aerolíneas vs retrasos promedio')
plt.xlabel('Aerolíneas')
plt.ylabel('Retraso promedio en minutos')
plt.show()

sns.countplot(data=datos, x='airline')
plt.title('Número de vuelos por aerolínea')
plt.xlabel('Aerolínea')
plt.ylabel('Número de vuelos')
plt.show()

NameError: name 'datos' is not defined

In [11]:
retraso_promedio = datos.groupby('schengen')['delay'].mean().reset_index()
sns.barplot(x='schengen', y='delay', data=retraso_promedio)
plt.title('Tipo de vuelo vs retrasos promedio')
plt.xlabel('Tipo de vuelo')
plt.ylabel('Retraso promedio en minutos')
plt.show()

sns.countplot(data=datos, x='schengen')
plt.title('Número de vuelos por tipo de vuelo')
plt.xlabel('Tipo de vuelo')
plt.ylabel('Número de vuelos')
plt.show()

NameError: name 'datos' is not defined

In [10]:
retraso_promedio = datos.groupby('is_holiday')['delay'].mean().reset_index()
sns.barplot(x='is_holiday', y='delay', data=retraso_promedio)
plt.title('Día festivo vs retrasos promedio')
plt.xlabel('¿Es día festivo?')
plt.ylabel('Retraso promedio en minutos')
plt.show()

NameError: name 'datos' is not defined

In [ ]:
orden = datos['aircraft_type'].value_counts().index
sns.countplot(data=datos, x='aircraft_type', order=orden)
plt.title('Número de vuelos por tipo de aeronave')
plt.xticks(rotation=70)
plt.xlabel('Tipo de aeronave')
plt.ylabel('Número de vuelos')
plt.show()

In [ ]:
import numpy as np

def calcular_ancho_bin(df, columna):
    Q75, Q25 = np.percentile(df[columna], [75, 25])
    IQR = Q75 - Q25
    ancho_bin = 2 * IQR * np.power(len(df[columna]), -1/3)
    return ancho_bin

In [ ]:
ancho_bin_at = calcular_ancho_bin(datos, 'arrival_time')
ancho_bin_dt = calcular_ancho_bin(datos, 'departure_time')
sns.histplot(data=datos, x='arrival_time', kde=True, binwidth=ancho_bin_at)
sns.histplot(data=datos, x='departure_time', kde=True, binwidth=ancho_bin_dt)

In [ ]:
media_retraso = datos['delay'].mean()
mediana_retraso = datos['delay'].median()

fig, axes = plt.subplots(1, 2, figsize=(9, 4))

sns.boxplot(data=datos, y='delay', ax=axes[0])
axes[0].set_title('Boxplot')

axes[0].axhline(y=media_retraso, color='r', linestyle='--', label='Media')
axes[0].legend()

ancho_bin_delay = calcular_ancho_bin(datos, 'delay')
sns.histplot(data=datos, x='delay', ax=axes[1], kde=True, binwidth=ancho_bin_delay)
plt.ylabel('Número de vuelos')
plt.grid(False)
axes[1].set_title('Histograma')

axes[1].axvline(x=media_retraso, color='r', linestyle='--', label='Media')
axes[1].axvline(x=mediana_retraso, color='y', linestyle='--', label='Mediana')
axes[1].legend()

plt.tight_layout()
plt.show()

# Mamos a la obra: Utilizando el modelo guardado

In [ ]:
nueva muestra = [0.0, 10.8941, ,0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [ ]:
schengen: 0
arrival_time: 10.8941
is_holiday: 0
is_weekend: 0
airline_BZ: 0
airline_MM: 0
airline_YE: 1
aircraft_type_Airbus A320: 1
aircraft_type_Airbus A330: 0
aircraft_type_Boeing 737: 0
aircraft_type_Boeing 777: 0
aircraft_type_Boeing 787: 0
aircraft_type_Embraer E175: 0


In [ ]:
import pickle

In [ ]:
file_path = './modelos/modelo_produccion.pkl'

try:
    with open(file_path, 'rb') as file:
        model = pickle.load(file)
    print("¡Modelo cargado con éxito!")
except Exception as e:
    print("Ocurrió un error al cargar el modelo:", str(e))


In [ ]:
model.predict([nueva_muestra])[0]

#Haga lo que hicimos

In [ ]:
from yellowbrick.model_selection import FeatureImportances

viz = FeatureImportances(model_rf, relative=False)
viz.fit(X_train, y_train)
viz.show()

In [ ]:
importances = model_rf.feature_importances_

feature_importances = pd.DataFrame({'Features': X.columns, 'Importances': importances})
feature_importances.sort_values('Importances', ascending=False)


In [ ]:
results_df = pd.DataFrame(index=['RMSE', 'MAE', 'R2'])

model_selected_features = RandomForestRegressor(random_state=42, max_depth=5)

for count in range(10, 16):
    selected_features = feature_importances['Features'].values[:count]

    X_train_selected = X_train[selected_features]
    X_test_selected = X_test[selected_features]

    model_selected_features.fit(X_train_selected, y_train)

    y_pred = model_selected_features.predict(X_test_selected)

    metricas = calcular_metricas_regressao(y_test, y_pred)

    results_df[count] = list(metricas.values())

results_df

In [ ]:
selected_features = feature_importances['Features'].values[:13]
X_select_features = X[selected_features]
X_train, X_test, y_train, y_test = train_test_split(X_select_features, y, random_state=42)


In [ ]:
param_grid = {
      'max_depth': [5, 10, 15],
      'min_samples_leaf': [1, 2, 3],
      'min_samples_split': [2, 4, 6],
      'n_estimators': [100, 150, 200]
 }

In [ ]:
from sklearn.model_selection import GridSearchCV

cv = KFold(n_splits=5, shuffle=True, random_state=42)

model_grid = GridSearchCV(RandomForestRegressor(random_state=42), param_grid=param_grid,
             scoring='r2', cv=cv)

model_grid.fit(X_train, y_train)

In [ ]:
y_pred_model_grid = model_grid.predict(X_test)
metricas_model_grid = calcular_metricas_regressao(y_test, y_pred_model_grid)
metricas_model_grid

In [ ]:
import pickle

try:
  with open('model_producao.pkl', 'wb') as file:
    pickle.dump(model_grid.best_estimator_, file)
  print('¡Modelo guardado con éxito!!!')
except Exception as e:
  print('Ocurrió un error al guardar el modelo: ', str(e))
